# Exercise 2: Benchmarking

In this exercise, you will implement benchmarking code for testing performance.

The workload will be the same vehicle detection code as exercise 1. 
But in this exercise, you will be repeating the process and timing it.


## Step 1: Inference Benchmark Scripts

In this step, you will be writing the benchmarking code for testing various hardware available to you on the DevCloud.
The hardware that you will test includes those that perform best when there are more than one request.
So as discussed in the video, we need to take advantage of the asynchronous inference mode to spawn the optimial number of inference requests.

For this exercise, you will be jumping straight to creating scripts for running the workload in the queue.
The scripts will be in two parts: `utils.py` where the helper functions like the function for peprocessing image are located, and `main.py` where the benchmarking occurs.

### utils.py

`utils.py` contains multiple helper functions for things like preprocessing. 
Follow the instructions to complete `utils.py`.

</br><details>
    <summary><b>(2.1)</b> Complete the <code>prepImage()</code> function by finding the NCHW values from the network.</summary>
    
Complete the `prepImage()` function by getting the values for `n`, `c`, `h` and `w` from the function input `net`.
The code here should be the exact same as in exercise 1.

</details><br/>

<details>
    <summary><b>(2.2)</b> Complete the <code>createExecNetwork()</code> function which takes IECore, IENetwork and device string and returns an ExecutableNetwork with the optimal number of requests.</summary>

To get the otimal number of requets, you first need a default ExecutableNetwork object. 
The IENetwork and device string is provided as input argument.
Use these along with IECore to get an ExecutableNetwork.

Then you can get the optimal number of requests from a metric of the ExecutableNetwork. 
See the slides for video 2 of course 2 for more details.
Use this value to recrate an ExecutablkeNetwork object with the optimal number of requests.
Finally, return this executable network.

</details>


In [1]:
%%writefile utils.py
import cv2
from openvino.inference_engine import IECore, IENetwork

def prepImage(input_layer,original_image, ie_net):

    ##! (2.1) Find n, c, h, w from net !##
    n, c, h, w = ie_net.inputs[input_layer].shape

    # Reshaping data
    input_image = cv2.resize(original_image, (w, h))
    input_image = input_image.transpose((2, 0, 1))
    input_image.reshape((n, c, h, w))

    return input_image

def getCount(detected_objects, prob_threshold=0.5):
    detected_count = 0
    for obj in detected_objects[0][0]:
        # Draw only objects when probability more than specified threshold
        if obj[2] > prob_threshold:
            detected_count+=1
    return detected_count

def createExecNetwork(ie_net, device):
    ##! (2.2) Create IECore !##
    ie = IECore()
    ##! (2.2) Create ExecutableNetwork object and find the optimal number of requests !##
    exec_net = ie.load_network(network=ie_net, device_name=device)
    nq = exec_net.get_metric("OPTIMAL_NUMBER_OF_INFER_REQUESTS")
    ##! (2.2) Recreate IECore and with num_requests set to optimal number of requests !##
    exec_net = ie.load_network(network=ie_net, device_name=device, num_requests=nq)
    ##! (2.2) return the ExecutableNetwork !##
    return exec_net

Overwriting utils.py


### main.py 
Next is the main.py. For this implementation, follow the approach where preprocessing and postprocessing are also repeated as many times as there are requests. Whilst it is not strictly necessary to repeat the preprocesing and postprocessing steps, it will give you timing that you can directly compare.

Follow the instructions to complete `main.py`.

</br><details>
    <summary><b>(2.3)</b> Create an IECore object and use it to cerate IENetwork object with the provded model. Then get the input and output layer names. Use <code>ie_core</code> and <code>ie_net</code> as the variable names.</summary>

The paths for the model is provided. Do not change the variable name, `ie_net` for ths file. The name of the input layer and output layer are stored in `inputs` and `outputs` dictionaries.

</details><br/>

<details>
    <summary><b>(2.4)</b> Start asynchronous processing on all request slots for images from <code>prepped_images</code>. </summary>

Asynchronous (non-blocking) inference is started with `start_async()`. A for loop over all the request slots are provided for you, so you just need to make sure that you use all the available request slots.

</details><br/>

<details>
    <summary><b>(2.5)</b> Wait for request slots to complete. </summary>

Use the `wait()` function with the requests to wait for the completion. You need to do this for all the slots.

</details><br/>

<details>
    <summary><b>(2.6)</b> Populate <code>result_list</code> with the number of vehicles detected from each slot. </summary>

Get the number of vehicles from each inference request with `getCount()` function, and save the result in `result_list`. Result of the inference itself can be accessed through the `outputs` attribute of the requests. See slides from course 1 video 7 for more. This array is used for a sanity check to make sure all inference requests return the same number of detected vehicles.

</details><br/>

In [2]:
%%writefile main.py
import time
from openvino.inference_engine import IENetwork, IECore
from utils import *
import cv2
import sys
import os
import statistics

# Getting the device as commandline argument
device = sys.argv[1]

##! (2.3) create IECore and IENetwork object for vehicle-detection-adas-0002 !##
xml_path="/data/intel/vehicle-detection-adas-0002/FP16/vehicle-detection-adas-0002.xml"
bin_path="/data/intel/vehicle-detection-adas-0002/FP16/vehicle-detection-adas-0002.bin"
# ie_core = None
ie_net = IENetwork(model=xml_path, weights=bin_path)


##! (2.3) get the input and output layer names !##
##! (2.3) get the input and output layer names !##
input_layer = next(iter(ie_net.inputs))
output_layer = next(iter(ie_net.outputs))

# Creating ExecutableNetwork object with the device variable for targtted device 
exec_net = createExecNetwork(ie_net, device)
num_requests = len(exec_net.requests)

image_path = "cars_1900_first_frame.jpg"
original_image = cv2.imread(image_path)

iter_ = 500
prep_time = []
infer_time = []
postp_time = []
for i in range(iter_):
    # Preprocessing image. 
    prep_start = time.time()
    prepped_images = []
    for slot_id in range(num_requests):
        prepped_images.append(prepImage(input_layer,original_image, ie_net))
    prep_time.append((time.time()-prep_start)/num_requests*1000)
    
    infer_start = time.time()
    for req_slot in range(num_requests):
        ##! (2.4) Run asynchronous inference. !##
        exec_net.start_async(request_id=req_slot, inputs={input_layer: prepped_images[req_slot]})
    for req_slot in range(num_requests):
        ##! (2.5) Wait for asynchronous inference to complete. !##
        exec_net.requests[req_slot].wait()
    infer_time.append((time.time()-infer_start)/num_requests*1000)
    
    postp_start = time.time()
    result_list = [0]*num_requests  # Python way of creating a 0 array of length 'num_requests'
    for req_slot in range(num_requests):
        ##! (2.6) Run getCount to get the vehicle count and store it in result_list !##
        result_list[req_slot] = getCount(exec_net.requests[req_slot].outputs[output_layer])
    postp_time.append((time.time()-postp_start)/num_requests*1000)

    # Sanity check to make sure all results are identical. Abort if it does not match
    assert all([x == result_list[0] for x in result_list]), "Results for the inference requests did not match"
    
# writing the results to a file
if not os.path.exists("results"):
    os.makedirs("results")
prep_avg    = statistics.mean(prep_time)
prep_stdev  = statistics.stdev(prep_time)
infer_avg   = statistics.mean(infer_time)
infer_stdev = statistics.stdev(infer_time)
postp_avg   = statistics.mean(postp_time)
postp_stdev = statistics.stdev(postp_time)
print("Inference running on: {}".format(device))
print("Number of requests: {}".format(num_requests))
print("Inference time per image (ms): {:.3g} +- {:.3g}".format(infer_avg, infer_stdev))
print("Preprocessing time per image (ms): {:.3g} +- {:.3g}".format(prep_avg, prep_stdev))
print("Postprocessing time per image (ms): {:.3g} +- {:.3g}".format(postp_avg, postp_stdev))

Overwriting main.py


## Step 3: Running Inference Benchmarks

With the benchmark scripts in hand you are ready to begin running benchmarks on the DevCloud.
The commands for running the job will be provided to you, just like in exercise 1.


With that said, there are some differences to note for the job submission in this exercise.
In exercise 1, the command to run the job was pushed to the `qsub` through the `echo` command. 
For this exercise, you will be passing the commands to run for job through a bash script.
The reason for this shift is that you will be using an FPGA machine for the benchmarks, and they require an additional step beyond executing `main.py`.
As discussed in the videos, FPGAs require "programs" in the form of bit-streams to be loaded.
For the `vehicle-detection-adas-0002` model, OpenVINO has a pre-built bit-stream for it.
So the commands have to be added to the bash script, and ran if FPGA is used.

### job file

Once again, the job file is provided for you. Note the if statement where we set up for FPGA if it is in the device list. Run the following cell to create the bash script `run.sh` to be used for benchmarking.

In [3]:
%%writefile run.sh

DEVICE=$1
source /opt/intel/openvino/bin/setupvars.sh

# Check if FPGA is used 
if grep -q FPGA <<<"$DEVICE"; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    export AOCL_BOARD_PACKAGE_ROOT=/opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2
    source /opt/altera/aocl-pro-rte/aclrte-linux64/init_opencl.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/2020-3_PL2_FP16_MobileNet_Clamp.aocx
    export CL_CONTEXT_COMPILER_MODE_INTELFPGA=3
fi
    
# Running the object detection code
python3 main.py $DEVICE

Overwriting run.sh


This bash script takes one argument, which specifies the device to use. 
The bit-stream is only loaded if "FPGA" appears in the device argument.

### Job queue submission

As in exercise 1, the command for submitting the job has been provided for you. 
Run the following cell to submit the job for processing with CPU. 

In [4]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "CPU", script_args=["CPU"], files=["cars_1900_first_frame.jpg","main.py","utils.py"])

Submitted. Job ID: L6aUuf7ZpCDlNN1iq3zmDFWW1iSto1Pc
Waiting for job to complete. This may take a few minutes.....

[setupvars.sh] OpenVINO environment initialized
Inference running on: CPU
Number of requests: 1
Inference time per image (ms): 13.3 +- 0.349
Preprocessing time per image (ms): 0.525 +- 1.32
Postprocessing time per image (ms): 0.413 +- 0.0234
main.py:16: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  ie_net = IENetwork(model=xml_path, weights=bin_path)



If the run on CPU was successful, it is time to try out the other devices.
Run the following cell to run the benchmark on GPU, FPGA and VPU.

**Note:** FPGA is set to `HETERO` mode with CPU, as there are some layers that are not supported by FPGA. For the VPU, we are using Intel Neural Compute Stick 2 (NCS2). 

In [5]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "GPU",  script_args=["GPU"], files=["cars_1900_first_frame.jpg","main.py","utils.py"])

Submitted. Job ID: odN8HWAUtUBYiRnmp94S5IiuHoW0TAZt
Waiting for job to complete. This may take a few minutes..............

[setupvars.sh] OpenVINO environment initialized
Inference running on: GPU
Number of requests: 2
Inference time per image (ms): 13.7 +- 0.464
Preprocessing time per image (ms): 0.436 +- 0.721
Postprocessing time per image (ms): 0.35 +- 0.0148
main.py:16: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  ie_net = IENetwork(model=xml_path, weights=bin_path)



Congratulations! You now have the performance benchmark on 4 types of device. 
Of course, these numbers are not the full story; you need consider other factors like power consumption and cost if these are important for your particular deployment.
But these benchmarks will be a key component in that decision making process.
**Take note of which device had the best (lowest) inference time per image value. This will be asked in the quiz**

In [6]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "FPGA", script_args=["HETERO:FPGA,CPU"], files=["cars_1900_first_frame.jpg","main.py","utils.py"])

Submitted. Job ID: lHlq2MDgx65QoRr0Zv09XhfLbUah8f5j
Waiting for job to complete. This may take a few minutes..........

[setupvars.sh] OpenVINO environment initialized
INTELFPGAOCLSDKROOT is not set
Using script's current directory (/opt/altera/aocl-pro-rte/aclrte-linux64)

aoc was not found, but aocl was found. Assuming only RTE is installed.

AOCL_BOARD_PACKAGE_ROOT is set to /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2. Using that.
Adding /opt/altera/aocl-pro-rte/aclrte-linux64/bin to PATH
Adding /opt/altera/aocl-pro-rte/aclrte-linux64/linux64/lib to LD_LIBRARY_PATH
Adding /opt/altera/aocl-pro-rte/aclrte-linux64/host/linux64/lib to LD_LIBRARY_PATH
Adding /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2/linux64/lib to LD_LIBRARY_PATH
aocl program: Running program from /opt/intel/openvino/bitstreams/a10_vision_design_sg2_bitstreams/BSP/a10_1150_sg2/linux64/libexec
Programming device: a10gx_2ddr : Intel Vision Accelera

In [7]:
from devcloud_utils import submitToDevCloud
submitToDevCloud("run.sh", "VPU",  script_args=["MYRIAD"], files=["cars_1900_first_frame.jpg","main.py","utils.py"])

Submitted. Job ID: UzkW4MtKnUKtiGEIcWQL7XQrNAbF0mR0
Waiting for job to complete. This may take a few minutes.....

[setupvars.sh] OpenVINO environment initialized
main.py:16: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  ie_net = IENetwork(model=xml_path, weights=bin_path)
Traceback (most recent call last):
  File "main.py", line 25, in <module>
    exec_net = createExecNetwork(ie_net, device)
  File "/submission/utils.py", line 28, in createExecNetwork
    exec_net = ie.load_network(network=ie_net, device_name=device)
  File "ie_api.pyx", line 178, in openvino.inference_engine.ie_api.IECore.load_network
  File "ie_api.pyx", line 187, in openvino.inference_engine.ie_api.IECore.load_network
RuntimeError: Can not init Myriad device: NC_ERROR

